In [152]:
import pandas as pd
import os


DATA_ROOT = os.path.join(os.getcwd(), 'data')

df_colors = pd.read_csv(os.path.join(
    DATA_ROOT, 'The Joy Of Painiting - Colors Used'))
df_colors['TITLE'] = df_colors.apply(
    lambda r: r['painting_title'].upper(), axis=1)
df_colors['EPISODE'] = df_colors.apply(
    lambda r: f"S{r['season']:02}E{r['episode']:02}", axis=1)

df_subjects = pd.read_csv(os.path.join(
    DATA_ROOT, 'The Joy Of Painiting - Subject Matter'))
# update column - remove surrounding quotes from TITLE
df_subjects['TITLE'] = df_subjects.apply(lambda r: r['TITLE'][1:-1], axis=1)

df_dates = pd.read_csv(os.path.join(
    DATA_ROOT, 'The Joy Of Painting - Episode Dates'), delimiter="\t", header=None, names=['src'])

# parse df_dates (not actually CSV-- parse each row of file as col in df)
df_dates['date'] = df_dates.apply(
    lambda r: r['src'][r['src'].find('(')+1:r['src'].find(')')].strip(), axis=1)
df_dates['title'] = df_dates.apply(
    lambda r: r['src'].split(r['date'])[0][:-1].strip(), axis=1
)
df_dates['note'] = df_dates.apply(
    lambda r: r['src'].split(r['date'])[1][1:-1].strip(), axis=1
)
df_dates['TITLE'] = df_dates.apply(lambda r: r['title'].upper(), axis=1)

# get complete TITLE -> EPISODE xwalk from union of matches in df_colors/df_subjects


df_titles = pd.concat([df_dates.set_index('TITLE', drop=False).join(
    df_colors[['TITLE', 'EPISODE']].set_index('TITLE'))[['TITLE', 'EPISODE']].reset_index(drop=True),
    df_dates.set_index('TITLE', drop=False).join(
    df_subjects[['EPISODE', 'TITLE']].set_index('TITLE'))[['TITLE', 'EPISODE']].reset_index(drop=True)])
# drop missing matches
df_titles.dropna(inplace=True)
# drop duplicates
df_titles.drop_duplicates(inplace=True)


In [180]:
# show missing titles
def getDatesForTitles(r):
    title = r['TITLE']
    matches = df_dates.loc[df_dates['TITLE'].str.contains(title)]
    return matches.iloc[0]['date']


def getEpisodeForDate(r):

    date = r['date']
    title = r['TITLE']
    matches = df_titles.loc[df_titles['date'].str.contains(date)]
    if matches.empty:
        matches = df_titles.loc[df_titles['TITLE'].str.contains(title)]
        if matches.empty:
            return None
    return matches.iloc[0]['EPISODE']


def getEpisodeXWalk(r):
    xwalk = {
        'EVERGREEN AT SUNSET': 'EVERGREENS AT SUNSET',
        'MOUNTAIN PASS': 'MOUNTAIN PASS',
        'SHADES OF GRAY': 'SHADES OF GREY',
        'EVENING SUNSET': 'EVENING AT SUNSET',
        'HALF OVAL VIGNETTE': 'HALF-OVAL VIGNETTE',
        'OLD OAK TREE': 'THE OLD OAK TREE',
        'CABIN AT TRIAL\'S END': 'THE CABIN AT TRIAL\'S END',
        'GREY MOUNTAIN': 'GRAY MOUNTAIN',
        'FOOTBRIDGE': 'THE FOOTBRIDGE',
        'OLD PLACE HOME': 'THE OLD HOME PLACE',
        'SUMMER IN THE MOUNTAINS': 'SUMMER IN THE MOUNTAIN',
        'WOODSMAN\'S RETREAT': 'WOODMAN\'S RETREAT',
        'PRETTY AUTUMN DAY': 'A PRETTY AUTUMN DAY',
        'MISTY FOREST': 'MISTY FOREST OVAL'
    }
    if r['EPISODE']:
        return r['EPISODE']
    title = r['TITLE']
    if title in xwalk:
        df_titles.loc[df_titles['TITLE'].str.contains(xwalk[r['TITLE']])]


df_titles['date'] = df_titles.apply(getDatesForTitles, axis=1)
df_dates['EPISODE'] = df_dates.apply(getEpisodeForDate, axis=1)
df_dates['EPISODE'] = df_dates.apply(getEpisodeXWalk, axis=1)


In [181]:


df_dates.loc[df_dates['EPISODE'].isnull()]


,src,date,title,note,TITLE,EPISODE
16,"Shades of Gray (September 21, 1983)","September 21, 1983",Shades of Gray,,SHADES OF GRAY,None
80,"Evergreen at Sunset (October 16, 1985)","October 16, 1985",Evergreen at Sunset,,EVERGREEN AT SUNSET,None
112,"Mountain Pass (June 25, 1986)","June 25, 1986",Mountain Pass,,MOUNTAIN PASS,None
159,"Evening Sunset (September 23, 1987)","September 23, 1987",Evening Sunset,,EVENING SUNSET,None
180,"Misty Forest (March 16, 1988)","March 16, 1988",Misty Forest,,MISTY FOREST,None
209,"Old Place Home (January 11, 1989)","January 11, 1989",Old Place Home,,OLD PLACE HOME,None
221,"Half Oval Vignette (July 5, 1989)","July 5, 1989",Half Oval Vignette,,HALF OVAL VIGNETTE,None
254,"Old Oak Tree (May 23, 1990)","May 23, 1990",Old Oak Tree,,OLD OAK TREE,None
264,"Cabin at Trial's End (October 3, 1990)","October 3, 1990",Cabin at Trial's End,,CABIN AT TRIAL'S END,None
299,"Grey Mountain (January 7, 1992)","January 7, 1992",Grey Mountain,,GREY MOUNTAIN,None


In [16]:
print("The Joy Of Painiting - Episode Dates:")
df_episode_dates.head()


The Joy Of Painiting - Episode Dates:


,0
0,"A Walk in the Woods (January 11, 1983)"
1,"Mount McKinley (January 11, 1983)"
2,"Ebony Sunset (January 18, 1983)"
3,"Winter Mist (January 25, 1983)"
4,"Quiet Stream (February 1, 1983)"
